In [7]:
!pip install datasets transformers[torch] soundfile librosa accelerate evaluate jiwer --quiet
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 73.0 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [11]:
!pip install -U transformers datasets accelerate peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 6.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.3.0
    Uninstalling accelerate-1.3.0:
      Successfully uninstalled accelerate-1.3.0
  Attempting uninstall: peft
    Found existing installation: peft 0.14.0
    Uninstalling peft-0.14.0:
      Successfully uninstalled peft-0.14.0


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==1

Lora Model And Training Arguments

In [8]:
!pip install git+https://github.com/huggingface/transformers.git accelerate datasets peft librosa --quiet

import torch
from datasets import load_dataset, Audio
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, PeftModel
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import os

dataset_path = "/kaggle/input/whisperlor/train-00000-of-00010.parquet"
dataset = load_dataset("parquet", data_files=dataset_path)["train"]

num_audio_files = len(dataset)
total_duration_sec = sum(audio["array"].shape[0] / audio["sampling_rate"] for audio in dataset["audio"])
print(f"Total Audio Files: {num_audio_files}")
print(f"Total Duration (hrs): {total_duration_sec / 3600:.2f}")

dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ["audio", "text"]])

modelname = "openai/whisper-tiny"
language = "hi"
task = "transcribe"

feature_extractor = WhisperFeatureExtractor.from_pretrained(modelname)
tokenizer = WhisperTokenizer.from_pretrained(modelname, language=language, task=task)
processor = WhisperProcessor.from_pretrained(modelname, language=language, task=task)

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]):
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

model = WhisperForConditionalGeneration.from_pretrained(
    modelname,
    device_map="auto",
    torch_dtype=torch.float32
)

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

#This  Create custom PeftModel class for Whisper
class WhisperPeftModel(PeftModel):
    def forward(self, input_features=None, **kwargs):
        return self.base_model(input_features=input_features, **kwargs)

model = WhisperPeftModel(model, config)
model.print_trainable_parameters()

# Custom trainer for Whisper
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        input_features = inputs.get("input_features")
        labels = inputs.get("labels")
        
        outputs = model(input_features=input_features, labels=labels)
        loss = outputs.loss
        
        return (loss, outputs) if return_outputs else loss

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-lora-hindi",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=6,
    fp16=False,
    generation_max_length=128,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    remove_unused_columns=False,
    label_names=["labels"],
    report_to=[],
)

# Save only LoRA adapter callback
class SavePeftModelCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        checkpoint_folder = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
        adapter_folder = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(adapter_folder)
        return control

trainer = CustomTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset,
    data_collator=data_collator,
    tokenizer=feature_extractor,
    callbacks=[SavePeftModelCallback()]
)

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

trainer.train()

model.save_pretrained("./whisper-lora-hindi-final")
processor.save_pretrained("./whisper-lora-hindi-final")

print("Training complete!")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Total Audio Files: 1183
Total Duration (hrs): 2.74
trainable params: 589,824 || all params: 38,350,464 || trainable%: 1.5380


/tmp/ipykernel_31/3728147076.py:137: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


Step,Training Loss
10,2.754700
20,2.098400
30,1.634300
40,1.430200
50,1.186800
60,0.998700
70,0.898200
80,0.710600
90,0.618200
100,0.524600


Training complete!


In [10]:
model.config.save_pretrained("whisper-lora-hindi-final")

In [11]:
!zip -r whisper-lora-hindi-final.zip whisper-lora-hindi-final/

!ls -lh *.zip

  adding: whisper-lora-hindi-final/ (stored 0%)
  adding: whisper-lora-hindi-final/special_tokens_map.json (deflated 80%)
  adding: whisper-lora-hindi-final/config.json (deflated 60%)
  adding: whisper-lora-hindi-final/normalizer.json (deflated 81%)
  adding: whisper-lora-hindi-final/merges.txt (deflated 54%)
  adding: whisper-lora-hindi-final/vocab.json (deflated 69%)
  adding: whisper-lora-hindi-final/adapter_config.json (deflated 54%)
  adding: whisper-lora-hindi-final/adapter_model.safetensors (deflated 7%)
  adding: whisper-lora-hindi-final/preprocessor_config.json (deflated 44%)
  adding: whisper-lora-hindi-final/added_tokens.json (deflated 80%)
  adding: whisper-lora-hindi-final/tokenizer_config.json (deflated 96%)
  adding: whisper-lora-hindi-final/README.md (deflated 66%)
-rw-r--r-- 1 root root 2.7M Apr 14 21:55 whisper-lora-hindi-final.zip


In [12]:
from IPython.display import FileLink
FileLink('whisper-lora-hindi-final.zip')  # Click to download

/kaggle/working/whisper-lora-hindi-final.zip